## Set-up

In [1]:
# Import packages 
from numpy import loadtxt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
import pandas as pd
import csv
import tensorflow as tf
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.callbacks import EarlyStopping#
import keras_tuner
import numpy as np
from sklearn.preprocessing import OneHotEncoder

## Dataset splitting and defining target and sample data

In [2]:
# Read in Training dataset
trn = pd.read_csv("df.csv", header = 0, na_filter = False)
trn

,Unnamed: 0,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted
0,1,72091308,20123568,Caucasian,Female,8,1,22,7,7,...,1,3,2,1,1,1,1,1,2,3
1,2,72848634,20377854,Caucasian,Female,7,2,1,1,3,...,1,3,2,1,1,1,1,2,2,3
2,3,73062156,20408121,Caucasian,Female,10,1,1,7,4,...,1,2,2,1,1,1,1,2,2,3
3,4,73731852,20542797,Caucasian,Male,8,1,2,7,10,...,1,3,2,1,1,1,1,1,2,3
4,5,81355914,7239654,Caucasian,Female,8,1,3,6,12,...,1,3,2,1,1,1,1,2,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26750,26751,443739044,106595208,Caucasian,Male,8,2,6,7,6,...,1,4,2,1,1,1,1,1,2,3
26751,26752,443793668,47293812,Caucasian,Male,9,1,13,7,3,...,1,1,2,1,1,1,1,1,2,3
26752,26753,443804570,33230016,Caucasian,Female,8,1,22,7,8,...,1,3,2,1,1,1,1,2,2,2
26753,26754,443816024,106392411,Caucasian,Female,8,3,6,1,3,...,1,3,2,1,1,1,1,1,2,3


In [3]:
# Check features in dataframe
print(list(trn))

['Unnamed: 0', 'encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id', 'time_in_hospital', 'payer_code', 'medical_specialty', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1', 'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'insulin', 'glyburide.metformin', 'glipizide.metformin', 'glimepiride.pioglitazone', 'metformin.rosiglitazone', 'metformin.pioglitazone', 'change', 'diabetesMed', 'readmitted']


In [4]:
# Drop redundant columns 
trn_new = trn.drop(trn.columns[[0]],
                       axis = 1)

In [5]:
trn_new = trn_new.drop(trn_new.columns[[0]],
                       axis = 1)

In [6]:
trn_new = trn_new.drop(trn_new.columns[[0]],
                       axis = 1)

In [7]:
# View the cleaned dataframe
trn_new.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,...,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted
0,Caucasian,Female,8,1,22,7,7,7,26,58,...,1,3,2,1,1,1,1,1,2,3
1,Caucasian,Female,7,2,1,1,3,7,18,59,...,1,3,2,1,1,1,1,2,2,3
2,Caucasian,Female,10,1,1,7,4,7,7,56,...,1,2,2,1,1,1,1,2,2,3
3,Caucasian,Male,8,1,2,7,10,7,17,68,...,1,3,2,1,1,1,1,1,2,3
4,Caucasian,Female,8,1,3,6,12,15,17,77,...,1,3,2,1,1,1,1,2,2,3


In [8]:
# Drop the predictor feature from the dataframe
trn_new2 = trn_new.drop(['gender'], axis = 1)

In [9]:
# View the sample dataframe
trn_new2

,race,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,...,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted
0,Caucasian,8,1,22,7,7,7,26,58,2,...,1,3,2,1,1,1,1,1,2,3
1,Caucasian,7,2,1,1,3,7,18,59,3,...,1,3,2,1,1,1,1,2,2,3
2,Caucasian,10,1,1,7,4,7,7,56,1,...,1,2,2,1,1,1,1,2,2,3
3,Caucasian,8,1,2,7,10,7,17,68,1,...,1,3,2,1,1,1,1,1,2,3
4,Caucasian,8,1,3,6,12,15,17,77,5,...,1,3,2,1,1,1,1,2,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26750,Caucasian,8,2,6,7,6,7,7,46,5,...,1,4,2,1,1,1,1,1,2,3
26751,Caucasian,9,1,13,7,3,7,7,45,0,...,1,1,2,1,1,1,1,1,2,3
26752,Caucasian,8,1,22,7,8,7,17,51,6,...,1,3,2,1,1,1,1,2,2,2
26753,Caucasian,8,3,6,1,3,7,25,27,1,...,1,3,2,1,1,1,1,1,2,3


In [10]:
# Sort the predictor variable
y1 = trn_new[['gender']]

In [11]:
# View the dataframe
y1

,gender
0,Female
1,Female
2,Female
3,Male
4,Female
...,...
26750,Male
26751,Male
26752,Female
26753,Female


In [12]:
# Check for unique values in race column
np.unique(trn_new['race'])

array(['AfricanAmerican', 'Asian', 'Caucasian', 'Hispanic', 'Other'],
      dtype=object)

In [13]:
# Creating instance of one-hot-encoder
enc = OneHotEncoder(handle_unknown='ignore')
# Passing race column to array
enc_df = pd.DataFrame(enc.fit_transform(trn_new2[['race']]).toarray())
# Merge with dataframe
trn_new2 = trn_new2.join(enc_df)
trn_new2

,race,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,...,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted,0,1,2,3,4
0,Caucasian,8,1,22,7,7,7,26,58,2,...,1,1,1,2,3,0.0,0.0,1.0,0.0,0.0
1,Caucasian,7,2,1,1,3,7,18,59,3,...,1,1,2,2,3,0.0,0.0,1.0,0.0,0.0
2,Caucasian,10,1,1,7,4,7,7,56,1,...,1,1,2,2,3,0.0,0.0,1.0,0.0,0.0
3,Caucasian,8,1,2,7,10,7,17,68,1,...,1,1,1,2,3,0.0,0.0,1.0,0.0,0.0
4,Caucasian,8,1,3,6,12,15,17,77,5,...,1,1,2,2,3,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26750,Caucasian,8,2,6,7,6,7,7,46,5,...,1,1,1,2,3,0.0,0.0,1.0,0.0,0.0
26751,Caucasian,9,1,13,7,3,7,7,45,0,...,1,1,1,2,3,0.0,0.0,1.0,0.0,0.0
26752,Caucasian,8,1,22,7,8,7,17,51,6,...,1,1,2,2,2,0.0,0.0,1.0,0.0,0.0
26753,Caucasian,8,3,6,1,3,7,25,27,1,...,1,1,1,2,3,0.0,0.0,1.0,0.0,0.0


In [14]:
# Drop original race column as its no longer needed
trn_new2_enc = trn_new2.drop(['race'], axis = 1)

## Early-Stopping

In [15]:
# Creating early stopping
custom_early_stopping = EarlyStopping(
    monitor='val_accuracy', 
    patience=20, 
    min_delta=0.001, 
    mode='max'
)

## Static Model

In [16]:
# Define the keras model
model = Sequential()
model.add(Input(shape=(45,)))
model.add(Dense(47, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [17]:
# Compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
# Change datatype for all int64 columns to float
s = trn_new2_enc.select_dtypes(include='int64').columns
trn_new2_enc[s] = trn_new2_enc[s].astype("float")

In [19]:
# Check for changes
trn_new2_enc[s].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26755 entries, 0 to 26754
Data columns (total 45 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       26755 non-null  float64
 1   admission_type_id         26755 non-null  float64
 2   discharge_disposition_id  26755 non-null  float64
 3   admission_source_id       26755 non-null  float64
 4   time_in_hospital          26755 non-null  float64
 5   payer_code                26755 non-null  float64
 6   medical_specialty         26755 non-null  float64
 7   num_lab_procedures        26755 non-null  float64
 8   num_procedures            26755 non-null  float64
 9   num_medications           26755 non-null  float64
 10  number_outpatient         26755 non-null  float64
 11  number_emergency          26755 non-null  float64
 12  number_inpatient          26755 non-null  float64
 13  diag_1                    26755 non-null  float64
 14  diag_2

In [20]:
# Converting dataframes to tensor
trn_new3 = tf.convert_to_tensor(trn_new2_enc[s])

In [21]:
# Create dictionary and map values to binary classes
gender_dict = {'Female':0, 'Male':1}
y1['gender'] = y1['gender'].map(gender_dict)
y1

C:\Users\benmk\AppData\Local\Temp\ipykernel_2904\904100397.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y1['gender'] = y1['gender'].map(gender_dict)


,gender
0,0
1,0
2,0
3,1
4,0
...,...
26750,1
26751,1
26752,0
26753,0


In [22]:
# View target column
y1

,gender
0,0
1,0
2,0
3,1
4,0
...,...
26750,1
26751,1
26752,0
26753,0


In [23]:
# Convert target column to tensor
y1 = tf.convert_to_tensor(y1)

In [24]:
# View tensor feature
y1

<tf.Tensor: shape=(26755, 1), dtype=int64, numpy=
array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]], dtype=int64)>

In [25]:
# Fit the keras model on the dataset
model.fit(trn_new3, y1, epochs=1500, batch_size=200, callbacks = [custom_early_stopping], validation_split=0.2)

Epoch 1/1500
108/108 [==============================] - 1s 3ms/step - loss: 5.1128 - accuracy: 0.4980 - val_loss: 1.6651 - val_accuracy: 0.5164
Epoch 2/1500
108/108 [==============================] - 0s 2ms/step - loss: 0.9513 - accuracy: 0.5199 - val_loss: 0.8071 - val_accuracy: 0.5208
Epoch 3/1500
108/108 [==============================] - 0s 2ms/step - loss: 0.7804 - accuracy: 0.5309 - val_loss: 1.1591 - val_accuracy: 0.5216
Epoch 4/1500
108/108 [==============================] - 0s 2ms/step - loss: 0.7960 - accuracy: 0.5294 - val_loss: 1.4795 - val_accuracy: 0.5229
Epoch 5/1500
108/108 [==============================] - 0s 2ms/step - loss: 0.8251 - accuracy: 0.5383 - val_loss: 0.7601 - val_accuracy: 0.5283
Epoch 6/1500
108/108 [==============================] - 0s 2ms/step - loss: 0.7413 - accuracy: 0.5406 - val_loss: 0.7714 - val_accuracy: 0.5070
Epoch 7/1500
108/108 [==============================] - 0s 2ms/step - loss: 0.7281 - accuracy: 0.5438 - val_loss: 0.7409 - val_accuracy:

## Model function

In [26]:
# Creating model function
def get_model(hp):
    model = Sequential()
    model.add(Input(shape=(45,)))
    for i in range(hp.Int("num_layers",1,7)):
        model.add(Dense(units = hp.Int(f"units_{i}",min_value = 10, max_value = 28, step = 3), activation = hp.Choice("activation",["relu","tanh"]),))
    model.add(Dense(1, activation='sigmoid'))
    learning_rate = hp.Float("lr",min_value = 1e-4,max_value = 1e-2,sampling="log")
    model.compile(loss = 'binary_crossentropy', optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate), metrics = ['accuracy'])
    return model

## Tensorboard

In [27]:
# Set tensorboard callback
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="C:/logs4")

## Keras Tuner

In [28]:
# Defining Keras tuner
tuner = keras_tuner.RandomSearch(hypermodel=get_model,objective="val_accuracy",max_trials=30,executions_per_trial=1,overwrite=True,directory="C:/Users/benmk/",project_name="diabetes4")

In [29]:
# Whats being captured by the Keras tuner
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 7, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 10, 'max_value': 28, 'step': 3, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [30]:
# Fit the keras model on the dataset
tuner.search(trn_new3, y1, epochs=1500, batch_size=200, callbacks = [custom_early_stopping, tensorboard_callback], validation_split=0.2)

Trial 30 Complete [00h 00m 12s]
val_accuracy: 0.5514857172966003

Best val_accuracy So Far: 0.5628854632377625
Total elapsed time: 00h 06m 33s
INFO:tensorflow:Oracle triggered exit


## Calling Tensorboard

In [31]:
# Load the extension for tensorboard
%load_ext tensorboard

In [33]:
# Reload extension for tensorboard
%reload_ext tensorboard

In [32]:
# Launch tensorboard
%tensorboard --logdir "C:/logs4"

Reusing TensorBoard on port 6006 (pid 20868), started 5 days, 4:24:34 ago. (Use '!kill 20868' to kill it.)

In [33]:
# Launch tensorboard alternative way
%tensorboard --logdir "C:/logs4"  --host localhost

Reusing TensorBoard on port 6006 (pid 23388), started 4 days, 9:38:14 ago. (Use '!kill 23388' to kill it.)